In [1]:
class ChatBot(object):
    """
    A conversational dialog chat bot.
    """

    def __init__(self, name, **kwargs):
        from .logic import MultiLogicAdapter

        self.name = name
        kwargs['name'] = name
        kwargs['chatbot'] = self

        self.default_session = None

        filters = kwargs.get('filters', tuple())
        self.filters = tuple([utils.import_module(F)() for F in filters])

        # Add required system logic adapter
        self.logic.system_adapters.append(
            utils.initialize_class('chatterbot.logic.NoKnowledgeAdapter', **kwargs)
        )

        for adapter in logic_adapters:
            self.logic.add_adapter(adapter, **kwargs)

        # Add the chatbot instance to each adapter to share information such as
        # the name, the current conversation, or other adapters
        self.logic.set_chatbot(self)
        self.input.set_chatbot(self)
        self.output.set_chatbot(self)

        preprocessors = kwargs.get(
            'preprocessors', [
                'chatterbot.preprocessors.clean_whitespace'
            ]
        )

        self.preprocessors = []

        for preprocessor in preprocessors:
            self.preprocessors.append(utils.import_module(preprocessor))

        # Use specified trainer or fall back to the default
        trainer = kwargs.get('trainer', 'chatterbot.trainers.Trainer')
        TrainerClass = utils.import_module(trainer)
        self.trainer = TrainerClass(self.storage, **kwargs)
        self.training_data = kwargs.get('training_data')

        self.default_conversation_id = None

        self.logger = kwargs.get('logger', logging.getLogger(__name__))

        # Allow the bot to save input it receives so that it can learn
        self.read_only = kwargs.get('read_only', False)

        if kwargs.get('initialize', True):
            self.initialize()

    def initialize(self):
        """
        Do any work that needs to be done before the responses can be returned.
        """
        self.logic.initialize()

    def get_response(self, input_item, conversation_id=None):
        """
        Return the bot's response based on the input.
        :param input_item: An input value.
        :param conversation_id: The id of a conversation.
        :returns: A response to the input.
        :rtype: Statement
        """
        if not conversation_id:
            if not self.default_conversation_id:
                self.default_conversation_id = self.storage.create_conversation()
            conversation_id = self.default_conversation_id

        input_statement = self.input.process_input_statement(input_item)

        # Preprocess the input statement
        for preprocessor in self.preprocessors:
            input_statement = preprocessor(self, input_statement)

        statement, response = self.generate_response(input_statement, conversation_id)

        # Learn that the user's input was a valid response to the chat bot's previous output
        previous_statement = self.storage.get_latest_response(conversation_id)

        if not self.read_only:
            self.learn_response(statement, previous_statement)
            self.storage.add_to_conversation(conversation_id, statement, response)

        # Process the response output with the output adapter
        return self.output.process_response(response, conversation_id)

    def generate_response(self, input_statement, conversation_id):
        """
        Return a response based on a given input statement.
        """
        self.storage.generate_base_query(self, conversation_id)

        # Select a response to the input statement
        response = self.logic.process(input_statement)

        return input_statement, response

    def learn_response(self, statement, previous_statement):
        """
        Learn that the statement provided is a valid response.
        """
        from .conversation import Response

        if previous_statement:
            statement.add_response(
                Response(previous_statement.text)
            )
            self.logger.info('Adding "{}" as a response to "{}"'.format(
                statement.text,
                previous_statement.text
            ))

        # Save the statement after selecting a response
        self.storage.update(statement)

    def set_trainer(self, training_class, **kwargs):
        """
        Set the module used to train the chatbot.
        :param training_class: The training class to use for the chat bot.
        :type training_class: `Trainer`
        :param \**kwargs: Any parameters that should be passed to the training class.
        """
        if 'chatbot' not in kwargs:
            kwargs['chatbot'] = self

        self.trainer = training_class(self.storage, **kwargs)

    @property
    def train(self):
        """
        Proxy method to the chat bot's trainer class.
        """
        return self.trainer.train

In [2]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import os

chatbot = ChatBot('Tora')
chatbot.set_trainer(ListTrainer)
# Train based on the english corpus
for _file in os.listdir('dataset'):
    if(_file != ".DS_Store" ):
        print(_file)
        chats = open('dataset/'+_file,'r').readlines()
        chatbot.train(chats)

ai.yml
List Trainer: [####################] 100%
botprofile.yml
List Trainer: [####################] 100%
computers.yml
List Trainer: [####################] 100%
conversations.yml
List Trainer: [####################] 100%
emotion.yml
List Trainer: [####################] 100%
food.txt
List Trainer: [####################] 100%
food.yml
List Trainer: [####################] 100%
gossip.yml
List Trainer: [####################] 100%
greetings.yml
List Trainer: [####################] 100%
history.yml
List Trainer: [####################] 100%
hotels.txt
List Trainer: [####################] 100%
humor.yml
List Trainer: [####################] 100%
literature.yml
List Trainer: [####################] 100%
money.yml
List Trainer: [####################] 100%
movies.yml
List Trainer: [####################] 100%
places.txt
List Trainer: [####################] 100%
politics.yml
List Trainer: [####################] 100%
psychology.yml
List Trainer: [####################] 100%
science.yml
List Trainer: [

In [3]:
# Get a response to an input statement
import time
while True:
    question = raw_input("You : ")
    if(question == "bye" or question == "Bye" or question == "exit" or question == "Exit"):
        print("Tora : Thanks for talking....")
        time.sleep(3)
        print("Tora : Hope you're satisfied :)")
        time.sleep(1)
        print("Tora : Bye then ...")
        time.sleep(2)
        break
        
    answer = chatbot.get_response(question)
    print("Tora : {}".format(answer))

You : hey
Tora : Hello master, how are you?
You : i'm fine
Tora : How can I help you today?
You : what are some sightseeing places in jaipur
Tora : Amer Fort, Hawa Mahal, Jantar Mantar, Nahargarh Fort, Jaigarh Fort, Sheesh Mahal, City Palace, Jal Mahal are the major places of interest and there are many more.
You : which of these is nearest to railway station
Tora : Jantar Mantar is the nearest to the railway station. It is at a distance of 5.2 km from the railway station.
You : i would like to stay in jaipur tonight
Tora : sure, i will show you some hotels.
You : what are some average price hotels
Tora : zone by park is one of good hotels out there.
You : find a hotel near to station
Tora : hotel star plaza, star palace, royal sunshine, ankur palace are some hotels near railway station.
You : i want to have some milkshakes
Tora : Keventers is the place you are looking for.
You : i want to have some good food in jaipur
Tora : Tell me about the type of food you want to have.
You : spicy